In [116]:
%matplotlib notebook

import matplotlib.pyplot as plt
import uuid
import pandas as pd
import numpy as np
from datetime import date, timedelta
import string

### Overview

Simulation model for sheep in a feedlot environment.  This simulation do not consider external management costs.

### Questions

- Profitability of an intensive sheep farming system
- Growth rates and alternative management and feeding schedules
- Sensitivity analysis of profitability to:

    - Birth rate (Fertility rate)
    - Mortality
    - Weaning rate = birth rate x mortality
    - Feed cost
    - Operation size (number of animals)
    
- Total feed cost per ewe per production cycle for different fertility rates
- Profit per ewe per production cycle
- Profit per year
- Monthly cash flow for different schedules
    
### Parameters

- Birth weight
- Birth rate (number of twins, triplets etc)
- Mortality rate (number of deaths per 1000 per year - lambs)
- Mortality rate (number of deaths per 1000 per year - ewes)
- Feed conversion Rate (time dependent model)
- Feed intake (nutritional requirements model for different growth stages)
- Growth Rate (Feed intake x FCR)
- Ram fertility

### Constants

- Pregnancy period
- Carcass weight as percentage of live weight

In [ ]:
class Schedule(object):
    
    def __init__(self, schedule):
        '''
        '''
        self.schedule = schedule

    def duration(self):
        # get the duration of the schedule
        
        keys = sorted(self.schedule)
        return keys[-1] - keys[0]

In [ ]:
SLP = 0.5 # Carcass weight as percentage of live weight

class FeedMix(object):
    
    # Energy content - metabolisable energy per kg (MJ / dry matter kg)
    # protein content
    # minerals and vitamins
    # moisture content - convert to DM
    def __init__(self, energy_pct, protein_pct, mass):
        self.DM = mass
        
    def cost_model(self):
        # cost of basic feedstuffs
        # calculate the cost of energy dependent on energy source
        # calculate the cost of protein dependent on protein source
        self.model = None

class Treatment(object):
    
    def __init__(self):
        '''
        '''

In [124]:
class GroupTypes:
    Ewes, Rams, Lambs = range(3)   
    
class Group(object):
    
    def __init__(self, name, grouptype):
        # name must be unique
        self.name = name
        self.sold = {}
        self.group_type = grouptype
        self.sheep = {}
        self.schedule_day = 0
        self.current_date = None
         
    def set_schedule_day(self, day, the_date):
        self.schedule_day = day
        self.current_date = the_date
        
    def count(self):
        return self.sheep.count
        
    def move(self, the_sheep, togroup):
        '''
        move the specified sheep from this group to the provided group
        '''
        self.remove(the_sheep)
        togroup.add(the_sheep)
    
    def add(self, the_sheep):
        self.sheep[the_sheep.id()] = the_sheep
    
    def remove(self, the_sheep):
        del self.sheep[the_sheep.id()]
        
    def sell(self, the_sheep):
        '''
        Sell the specified sheep
        '''
        
        self.sold[the_sheep.id] = the_sheep
        
    def mate(self, rams):
        '''
        mate the ewes in this group with the provided rams
        '''
        
    def get_num_required_rams(self):
        '''
        gives the required number of rams needed to mate all the ewes in this group in a single day (24 hour period)
        '''
 
    def feed(self, date):
        '''
        increment age
        feed the required rations
        '''
        
        
    def avg_weight(self):
        '''
        calculate the average weight of the group
        '''
        
        return np.mean([s.weight() for s in self.sheep()])
        
    def sheep(self):
        return sheep.values
    


In [142]:
class Sheep(object):
    
    def __init__(self, group_id, animal_type, age = 0):
        self.id = uuid.uuid4()
        self.group_id = group_id;
        self.animal_type = animal_type
        
        self.weight = 0.5 # average birth weight
        self.age = age # days
        
        self.medicine = pd.DataFrame()
        self.feed = pd.DataFrame()
        self.cost = pd.DataFrame()
        
        self.value = 0.0
        self.profit = 0.0
        self.status = "Healthy" # either healthy, sick, dead
        
    def feed(self, feedmix, quantity):
        self.feed[date] = feed
        # weight increases according to FCR - feed conversion rate
        # add to cost table
    
    def medicate(self, treatment):
        '''
        Apply the treatment
        '''
    
    def weight(self):
        return self.weight
    
    def sell(self, price_kg, cost):
        # sell the sheep
        self.value = self.weight * SLP * price_kg - cost
        self.profit = self.value - self.acc_cost()
        
    def acc_cost(self):
        '''
        calculate the accumulated cost
        ''' 
        return self.cost.sum()
        
    def profit(self):
        # sales value minus accumulated cost in Rands
        return self.profit
    
    def group_id(self):
        return self.group_id;
    
    def die(self):
        '''
        the animal dies
        '''
    
class Ram(Sheep):
    def __init__(self):
        super(Ewe, self).__init__("Ram")
        
class Lamb(Sheep):
    def __init__(self, mother, father, *args):
        super(Sheep, self).__init__("Lamb")
        self.sire = father
        self.dam = mother
        # siblings are specified as *args
    
    def acc_cost(self):
        '''
        The accumulated cost of a lamb is that of himself and his share of his mother's feed.
        '''
    
class Ewe(Sheep):
    '''
    The accumulated cost of a ewe is cleared whenever her lamb(s) get sold if they can cover their own cost and her cost, 
    if she skips lambing, her cost continues to accumulate until her lamb(s) are sold or she are sold.  When a ewe dies
    her accumulated cost is substracted from the farm account:    
    
    '''
    lambs = {}
    def __init__(self):
        super(Ewe, self).__init__("Ewe")
    
    def lamb(self):
        # dependent on 
        newborn = Lamb()
        lambs[newborn.id] = newborn
        
    def get_lambs(self):
        '''
        '''
        return lambs.values()
        

**Management schedule definition: **

Defines the dates to take certain actions

Document format:

day:grouptype:action

**Action Examples:**

mate ewes
Inocculate with xyz
Dose for worms
Scan for pregnancy
Weigh
Move Ewes that are not pregnant to other group
Wean lambs
Cull ewes
Slaughter lambs
Prepare ewes
Prepare rams

**Feed schedule definition: **

Document format:

day:mix:feedingrate

Defines the dates the feeding mix and/or consumption changes

A feeding schedule is defined for a specific group of animals all with the same characteristics:

Rams in a group
Ewes in a group
Lambs in a group


In [143]:
actions = {
    "prepare_ewes" : lambda g: print("preparing group: " + g.name) if (g.group_type==GroupTypes.Ewes) else None,
    "prepare_rams" : lambda g: print("preparing group: " + g.name) if (g.group_type==GroupTypes.Rams) else None,
    "END" : lambda g, day: g.set_schedule_day(day)
}

class ActionMap(object):
    
    def __init__(self, actions):
        '''
        '''
        self.actions = actions
        
    def get_action(self, key):
        '''
        get the lambda defined for the key
        '''
        
        return actions[key]
    

        
# day : (group type, actions)
schedule = {
    0: (GroupTypes.Rams, "prepare_rams"),
    30: (GroupTypes.Ewes, "mate_ewes"),
    100: (GroupTypes.Ewes, "scan_ewes"),
    240: (GroupTypes.Ewes, "END")
}

In [144]:
test = Schedule(schedule)
test.duration()

240

**Lambing System**

#number of groups
lambing cycle period
lag between groups schedules = lambing cycle period / #number of groups
Ewe/Rams ratio

- Monthly lambing
- Bi-monthly lambing

In [145]:
class SheepSystem(object):
    
    def __init__(self, numanimals, numgroups, cycleperiod):
        '''
        '''
        self.num_sheep = numanimals
        self.num_groups = numgroups
        self.cycle_period = cycleperiod * 30
        
    def set_management_schedule(self, schedule):
        self.management_schedule = schedule
    
    def set_feeding_schedules(self, ramsched, ewesched, lambsched):
        self.ram_feeding_schedule = ramsched
        self.ewe_feeding_schedule = ewesched
        self.lamb_feeding_sched = lambsched        

class Account(object):
    '''
    The financial account for a farm
    '''
    
    def __init__(self, capital):
        '''
        
        '''
        
class Farm(object) :
    
    def __init__(self, sheepSystem, capital):
        self.system = sheepSystem
        self.feed = {}
        self.sales = {}
        self.account = Account(capital)
        
        # create the animals
        
        # create the groups
        self.groups = self._create_groups(self.system.num_groups)
        self.yesterday = None
     
        # create the lag for each group according to the system spec
        self.group_schedules = self._create_group_scheduling()
        self.day_count = 0
    
    def _create_groups(self, numgroups):
        grps = {}
        
        for name in list(string.ascii_uppercase)[:numgroups]:
            grps[name] = Group(name, GroupTypes.Ewes)
        return grps       
        
    def _create_group_scheduling(self):
        schedule_days = {}
        days_lag = self.system.cycle_period / self.system.num_groups
        lag = 0
        for g in self.get_groups():
            schedule_days[g.name] = 0 - lag
            lag += days_lag
            
        return schedule_days
    
    def group_schedule_day(self, group_name, glob_day):
        '''
        '''

        return self.group_schedules[group_name] + glob_day
    
    def get_groups(self):
        '''
        '''
        return self.groups.values()
    
    def max_schedule_duration(self):
        '''
        '''
    
    def new_day(self, date):
        # apply the actions for each schedule in order
        # management schedule
        if self.yesterday is None:
            print("Start farming...!! \n")
            self.day_count = 0
        
        print(str(date) + ": What should we do today: ")
        print("  MANAGEMENT: ")
        
        # for each group - see what should we do today"
        for g in self.get_groups():
            print("\t" + g.name + ": " + " Day: " + str(self.group_schedule_day(g.name, self.day_count)))
            
        # feeding schedules - rams, ewes, lambs
        
        # do the tasks for the day 
        
        # when the ewes are prepared for lambing or start lambing create a lamb group
        
        # a group is removed when no animals exist in the group
        
        # check for deaths
        
        # lekker slaap
        self.yesterday = date
        self.day_count += 1
        
    def get_sales(self):
        '''
        '''
        # time series of sales
        # date, number of animals, avg weight, price/kg, value
        
    def get_feed(self):
        '''
        '''
        # time series of feed
        # date, grouptype, feedmix, number of animals, avg weight, kg feed, price/kg, value
    
    def get_profit(self):
        '''
        '''
        # calculated at time of sale. Sales minus accumulated cost per animal.
        # date, number of animals, total cost, total sales value, total profit, price_per_sheep, cost_per_sheep, profit_per_sheep
    
class Simulation(object):
    
    def __init__(self, system, capital):
        # create a farm
        self.farm = Farm(system, capital)
       
    def run(self, startdate, enddate):
        delta = enddate - startdate
        calendar = [startdate + timedelta(days=x) for x in range(delta.days)]
        for date in calendar:
            self.farm.new_day(date)
                                

In [146]:
# 300 ewes, 8 groups, 8 month lamb cycle
system = SheepSystem(300, 8, 8)
# run the above system with R100k starting capital
sim = Simulation(system, 100000)
sim.run(date.today(), date.today() + timedelta(1000))

Start farming...!! 

2016-08-19: What should we do today: 
  MANAGEMENT: 
	E:  Day: 0
	H:  Day: -30.0
	B:  Day: -60.0
	G:  Day: -90.0
	A:  Day: -120.0
	D:  Day: -150.0
	F:  Day: -180.0
	C:  Day: -210.0
2016-08-20: What should we do today: 
  MANAGEMENT: 
	E:  Day: 1
	H:  Day: -29.0
	B:  Day: -59.0
	G:  Day: -89.0
	A:  Day: -119.0
	D:  Day: -149.0
	F:  Day: -179.0
	C:  Day: -209.0
2016-08-21: What should we do today: 
  MANAGEMENT: 
	E:  Day: 2
	H:  Day: -28.0
	B:  Day: -58.0
	G:  Day: -88.0
	A:  Day: -118.0
	D:  Day: -148.0
	F:  Day: -178.0
	C:  Day: -208.0
2016-08-22: What should we do today: 
  MANAGEMENT: 
	E:  Day: 3
	H:  Day: -27.0
	B:  Day: -57.0
	G:  Day: -87.0
	A:  Day: -117.0
	D:  Day: -147.0
	F:  Day: -177.0
	C:  Day: -207.0
2016-08-23: What should we do today: 
  MANAGEMENT: 
	E:  Day: 4
	H:  Day: -26.0
	B:  Day: -56.0
	G:  Day: -86.0
	A:  Day: -116.0
	D:  Day: -146.0
	F:  Day: -176.0
	C:  Day: -206.0
2016-08-24: What should we do today: 
  MANAGEMENT: 
	E:  Day: 5
	H:  Day: